In [1]:
# importing libraries
import pandas as pd
import numpy as np
import statistics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas_profiling as pp

In [2]:
products = pd.read_csv("products.csv")

In [3]:
# pandas profiling will do the basic eda of the dataset and gives you the results as follows

# it  gives us idea whether the column is categorical or numerical
# it gives us an idea whether the column has missing values or not
# it gives correlation between the numerical columns

# A sample can also be viewed from the given dataset
pp.ProfileReport(products)

Number of variables,11
Number of observations,7226
Total Missing (%),3.1%
Total size in memory,621.1 KiB
Average record size in memory,88.0 B
Numeric,1
Categorical,8
Boolean,0
Date,0
Text (Unique),1
Rejected,1


In [4]:
products.columns

Index(['ProductId', 'Title', 'Image URL', 'Division', 'Category',
       'Sub category', 'Article type', 'Offer price', 'Sale price', 'Color',
       'Gender'],
      dtype='object')

In [5]:
products.isnull().sum()

ProductId          0
Title              0
Image URL          0
Division        2394
Category           0
Sub category       0
Article type       0
Offer price        0
Sale price         0
Color            108
Gender             0
dtype: int64

In [6]:
products['Color'].mode()

0    Crunchy Fashion
dtype: object

In [7]:
# imputing null values based on mode for color as it has only 1.5 percent missing values
products['Color'].fillna(products['Color'].mode()[0], inplace=True)

In [8]:
# dropping the column from similarity matrix as it has around 33 percent missing values
products = products.drop(['Division'],axis=1)

In [9]:
# verifying whether all the null data is imputed
products.isnull().sum()

ProductId       0
Title           0
Image URL       0
Category        0
Sub category    0
Article type    0
Offer price     0
Sale price      0
Color           0
Gender          0
dtype: int64

In [10]:
# using Term Frequency inverse document frequency vectorizer to create a sparse matrix

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(products['Title'])

In [11]:
#shape of the sparse matrix created after using TFIDF vectorizer
tfidf_matrix

<7226x51512 sparse matrix of type '<class 'numpy.float64'>'
	with 149534 stored elements in Compressed Sparse Row format>

In [12]:
#creating Cosine similarities between the products
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in products.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], products['ProductId'][i]) for i in similar_indices]
    
    
    results[row['ProductId']] = similar_items[1:]

In [13]:
def item(ProductId):
    return products.loc[products['ProductId'] == ProductId]['Title'].tolist()[0].split(' - ')[0]

In [14]:
# recommendations based on productid and 
def recommend(item_id, num):
    print('Recommendations are:')
    recs = results[item_id][:num]
    for rec in recs:
        print(item(rec[1]))

In [16]:
# printing input item name and displaying the results as per the given input
print("name of the selected product is ",products[products["ProductId"]=="53D8EC1C829A67AF16E54FB35930FCC3"].Title)
recommend(item_id='53D8EC1C829A67AF16E54FB35930FCC3', num=10)

name of the selected product is  190    FACES Ultime Pro Longwear Lipstick Fig 08 (2.5 g)
Name: Title, dtype: object
Recommendations are:
FACES Ultime Pro Longwear Lipstick Secret Affair 17 (2.5 g)
FACES Ultime Pro Eyebrow Pencil (1.2 g)
FACES Ultime Pro Waterproof Mascara (7.5 ml)
FACES Ultime Pro Waterproof Mascara (7.5 ml)
FACES Ultime Pro Concealer Light (3.2 ml)
FACES Ultime Pro Concealer Light (3.2 ml)
FACES Ultime Pro Concealer Medium (3.2 ml)
Faces Go Chic Lipstick Fig 112
FACES Ultime Pro Eyeshadow Crayon Mamma Mia 08 1.(6 g)
Faces Ultime Pro Lip Creme Peach On The Beach 08 (4.6 ml)
